In [1]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import psycopg2
from psycopg2 import sql

# Load environment variables from .env file
load_dotenv()

# Database connection setup
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

db_url = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(db_url)

# Fetch symbols from the Asset table in PostgreSQL
symbols = []
with engine.connect() as conn:
    result = conn.execute(text("SELECT AssetCode FROM Asset WHERE AssetStatus = 'Active'"))
    for row in result:
        symbols.append(row[0])

# Fetch and process data for each symbol
all_data = []

for symbol in symbols:
    try:
        # Download stock data for each symbol
        df = yf.download(tickers=symbol, period="5d", interval="1d")
        
        # If no data is returned, skip this symbol
        if df.empty:
            print(f"No data for symbol: {symbol}")
            continue

        # Keep only relevant columns, rename, and add Symbol column
        df = df[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']].copy()
        df.columns = ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']
        df['Symbol'] = symbol
        df.reset_index(inplace=True)  # Flatten the Date index

        all_data.append(df)

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Combine all data into a single DataFrame with uniform columns
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Display a sample to verify data structure
    print("Sample data from combined DataFrame:")
    print(combined_df.head())

    # Create the temporary table 'pricestaging_temp'
    with engine.connect() as conn:
        conn.execute(text("""
            CREATE TEMPORARY TABLE IF NOT EXISTS prctemp (
                Date DATE,
                Symbol VARCHAR,
                Adj_Close NUMERIC,
                Close NUMERIC,
                High NUMERIC,
                Low NUMERIC,
                Open NUMERIC,
                Volume INTEGER
            );
        """))
        
        # Insert data into the temporary table 'pricestaging_temp'
        combined_df.to_sql('prctemp', con=engine, if_exists='replace', index=False)

    # Now move the data from the temporary table to the final table
    with psycopg2.connect(
        dbname=db_name, user=db_user, password=db_password, host=db_host
    ) as conn:
        with conn.cursor() as cursor:
            # SQL for upserting data from temporary table to the final price table
            cursor.execute("""
                INSERT INTO public.price (assetid, currencycode, amount, pricedate, datecreated, dateupdated)
                SELECT 
                    a.assetid,
                    'USD'::varchar(3),  -- Assuming USD is the currency for all assets
                    pst."Close"::numeric(18, 5),  -- Using 'Close' from the temporary table
                    pst."Date"::timestamp,  -- Using 'Date' from the temporary table
                    CURRENT_TIMESTAMP,  -- Date created
                    CURRENT_TIMESTAMP  -- Date updated
                FROM (
                    -- Deduplicate the temporary table based on symbol and date
                    SELECT DISTINCT ON (pst."Symbol", pst."Date") 
                        pst."Close", 
                        pst."Date", 
                        pst."Symbol"
                    FROM prctemp pst
                    ORDER BY pst."Symbol", pst."Date" DESC  -- Ensure the latest record is picked for each Symbol and Date
                ) pst
                JOIN public.asset a ON pst."Symbol" = a.assetcode
                ON CONFLICT (assetid, pricedate) 
                DO UPDATE 
                SET 
                    amount = EXCLUDED.amount,
                    dateupdated = CURRENT_TIMESTAMP;  -- Update the amount and dateupdated if the record already exists
            """)

            # Commit the transaction
            conn.commit()

    print("Data fetch, temporary table creation, and insertion complete.")
else:
    print("No data fetched. Exiting.")


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['USDCASH']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')


No data for symbol: USDCASH


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GBPCASH']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


No data for symbol: GBPCASH


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

No data for symbol: CRSRDIDI


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_46657/3260464532.py:62: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(text("""


Sample data from combined DataFrame:
                       Date  Adj_Close      Close       High        Low  \
0 2024-11-15 00:00:00+00:00  35.000000  35.000000  35.195000  33.660000   
1 2024-11-18 00:00:00+00:00  35.330002  35.330002  35.700001  35.060001   
2 2024-11-19 00:00:00+00:00  35.189999  35.189999  35.189999  34.500000   
3 2024-11-20 00:00:00+00:00  35.180000  35.180000  35.490002  34.880001   
4 2024-11-21 00:00:00+00:00  35.380001  35.380001  35.759998  35.099998   

        Open    Volume Symbol  
0  34.250000  17140200     JD  
1  35.430000  10651100     JD  
2  34.599998   8723000     JD  
3  35.480000   8294500     JD  
4  35.180000   9102000     JD  
Data fetch, temporary table creation, and insertion complete.
